In [1]:
import os
import shutil
import pathlib
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from ops.split_data import SplitDataset
from lib.SpectraData import *
import ops.trains as trains
import ops.tests as tests
from models.vit import *

In [2]:
shutil.rmtree('./data/Reflective_animalBlood_preprocessing/test')
os.mkdir('./data/Reflective_animalBlood_preprocessing/test')
shutil.rmtree('./data/Reflective_animalBlood_preprocessing/train')
os.mkdir('./data/Reflective_animalBlood_preprocessing/train')
shutil.rmtree('./data/Reflective_animalBlood_preprocessing/valid')
os.mkdir('./data/Reflective_animalBlood_preprocessing/valid')

In [3]:
split_dataset = SplitDataset(dataset_dir='/Users/renpengju/Documents/python_codes/experience_code/Data_set/Reflective_animalBlood_preprocessing', saved_dataset_dir='./data/Reflective_animalBlood_preprocessing', show_progress=False)
split_dataset.start_splitting()

In [4]:
spectra_path = './data/Reflective_animalBlood_preprocessing'

train_dataset = SpectraData(spectra_path, mode='train')
valid_dataset = SpectraData(spectra_path, mode='valid')
test_dataset = SpectraData(spectra_path, mode='test')

print(train_dataset)
print(valid_dataset)
print(test_dataset)

Finished reading the train set of Spectra Dataset (3131 samples found)
Finished reading the valid set of Spectra Dataset (1059 samples found)
Finished reading the test set of Spectra Dataset (1039 samples found)


In [5]:
# 定义dataloader
train_dataloader = DataLoader(dataset = train_dataset, batch_size=128, shuffle=True, num_workers = 0,)
valid_dataloader = DataLoader(dataset = valid_dataset, batch_size=128, shuffle=True, num_workers = 0,)
test_dataloader = DataLoader(dataset = test_dataset, batch_size=128, shuffle=False, num_workers = 0,)

In [6]:
from torch.utils.tensorboard import SummaryWriter

model = ViT(spectra_size=1400, patch_size=140, num_classes=40, dim=512, depth=8, heads=16, dim_mlp=1400, channel=1, dim_head=32)
model

ViT(
  (embedding): Sequential(
    (0): PatchEmbdding(
      (patch_embedding): Sequential(
        (0): Rearrange('b c (d p) -> b d (p c)', p=140)
        (1): Linear(in_features=140, out_features=512, bias=True)
      )
    )
    (1): CLSToken()
    (2): AbsPosEmbedding()
    (3): Identity()
  )
  (transformers): Sequential(
    (0): Transformer(
      (shortcut): Sequential()
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attn): Attention1d(
        (to_qkv): Linear(in_features=512, out_features=1536, bias=False)
        (to_out): Sequential(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): Identity()
        )
      )
      (sd1): Identity()
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (ff): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=512, out_features=1400, bias=True)
          (1): GELU(approximate='none')
          (2): Identity()
          (3): Linear(in_f

In [7]:
import datetime

current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
log_dir = os.path.join("runs", 'Raman_spectra', 'ViT', current_time)
writer = SummaryWriter(log_dir)

print('Create TensorBoard log dir: ', log_dir)

Create TensorBoard log dir:  runs/Raman_spectra/ViT/20230222_193644


In [8]:
device = torch.device('mps')
device

device(type='mps')

In [9]:
import ops.schedulers as schedulers

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200, eta_min=0, )
warmup_scheduler = schedulers.WarmupScheduler(optimizer, len(train_dataloader) * 10)

trains.train(model=model, train_iter=train_dataloader, valid_iter=valid_dataloader, optimizer=optimizer, train_schedule=scheduler, warmup_scheduler=warmup_scheduler, device=device, writer=writer)

training on:  mps
(7.31 sec/epoch) Warmup Epoch: 0, Loss: 3.3742, Acc: 0.0898, Precision: 0.0655, Recall: 0.8937, F1_score: 0.0270, lr: 1.000e-04
Valid Dataset: , Loss: 2.9300, Acc: 0.1487, Precision: 0.1429, Recall: 0.7945, F1: 0.0669
(6.66 sec/epoch) Warmup Epoch: 1, Loss: 2.4870, Acc: 0.2391, Precision: 0.2013, Recall: 0.7474, F1_score: 0.1342, lr: 2.000e-04
Valid Dataset: , Loss: 2.0088, Acc: 0.3679, Precision: 0.3436, Recall: 0.6780, F1: 0.2508
(6.71 sec/epoch) Warmup Epoch: 2, Loss: 1.7342, Acc: 0.4253, Precision: 0.4043, Recall: 0.6991, F1_score: 0.3133, lr: 3.000e-04
Valid Dataset: , Loss: 1.3655, Acc: 0.5083, Precision: 0.5231, Recall: 0.6912, F1: 0.4109
(6.69 sec/epoch) Warmup Epoch: 3, Loss: 1.3084, Acc: 0.5451, Precision: 0.5430, Recall: 0.7160, F1_score: 0.4472, lr: 4.000e-04
Valid Dataset: , Loss: 1.0902, Acc: 0.6398, Precision: 0.6455, Recall: 0.7415, F1: 0.5496
(6.68 sec/epoch) Warmup Epoch: 4, Loss: 1.0322, Acc: 0.6296, Precision: 0.6334, Recall: 0.7289, F1_score: 0.54

In [10]:
model = torch.load('./save_weight/reflect_model.pkl')
model

ViT(
  (embedding): Sequential(
    (0): PatchEmbdding(
      (patch_embedding): Sequential(
        (0): Rearrange('b c (d p) -> b d (p c)', p=140)
        (1): Linear(in_features=140, out_features=512, bias=True)
      )
    )
    (1): CLSToken()
    (2): AbsPosEmbedding()
    (3): Identity()
  )
  (transformers): Sequential(
    (0): Transformer(
      (shortcut): Sequential()
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attn): Attention1d(
        (to_qkv): Linear(in_features=512, out_features=1536, bias=False)
        (to_out): Sequential(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): Identity()
        )
      )
      (sd1): Identity()
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (ff): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=512, out_features=1400, bias=True)
          (1): GELU(approximate='none')
          (2): Identity()
          (3): Linear(in_f

In [11]:
tests.test(model, test_dataloader, device=device)

Classification_report               precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000        20
         1.0     1.0000    1.0000    1.0000         6
         2.0     0.9375    1.0000    0.9677        30
         3.0     0.9661    0.9828    0.9744        58
         4.0     0.9508    0.9831    0.9667        59
         5.0     1.0000    1.0000    1.0000        39
         6.0     1.0000    1.0000    1.0000         6
         7.0     0.9796    0.9600    0.9697        50
         8.0     1.0000    1.0000    1.0000        59
         9.0     1.0000    1.0000    1.0000        30
        10.0     1.0000    0.8929    0.9434        28
        11.0     1.0000    1.0000    1.0000        30
        12.0     0.9091    1.0000    0.9524        30
        13.0     1.0000    1.0000    1.0000         6
        14.0     1.0000    1.0000    1.0000         6
        15.0     1.0000    1.0000    1.0000         6
        16.0     1.0000    1.0000    1.0000        29
     